In [6]:
import os
import re
import urllib.request
from bs4 import BeautifulSoup

def crawler():
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    url = 'https://gg12.ru/category/stati/page/%s'
    craw_list = []
    for i in range(1, 29):
        req = urllib.request.Request(url % i, headers={'User-Agent':user_agent})
        with urllib.request.urlopen(req) as response:
            html = response.read().decode('utf-8')
        artcls_lnks = re.findall('<article .+?href="[^"]+"', html, re.DOTALL)
        for lnk in artcls_lnks:
            link = re.search('href="([^"]+)"', lnk).group(1)
            craw_list.append(link)
    return craw_list

In [7]:
#def mystem(content, date, file_number):
#    print('yes')
#    os.system('cd /Users/new.aloha/Downloads/')
#    os.system("% echo '" + content + "' | ./mystem - output.txt")

In [8]:
def get_ready():
    root = 'Газета Йошкар-Ола'
    if not os.path.exists(root):
        os.makedirs(root)
    tab = root + os.sep + 'meta.csv'
    line = 'path\tauthor\theader\tcreated\t'
    line += 'sphere\ttopic\tstyle\taudience_age\t'
    line += 'audience_level\taudience_size\tsource\t'
    line += 'publication\tpubl_year\tmedium\t'
    line += 'country\tregion\tlanguage\n'
    if not os.path.exists(tab):
        f = open(tab, 'w', encoding = 'utf-8')
        f.write(line)
        f.close()
    return tab

In [9]:
def page(url, file_number, tab):
    root = 'Газета Йошкар-Ола'
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req) as response:
        html = response.read().decode('UTF-8')
    beginning = html.find('<div class="entry-content">')
    beginning = html.find('<p>', beginning)
    end = html.find('<div class="post-views')
    content = html[beginning:end]
    soup = BeautifulSoup(content, 'html.parser')
    content = soup.get_text()
    for i in range(len(content)):
        if content[i] == '.':
            content = content[:i+1] + '\n' + content[i+1:]
            # каждое предложение на новой строчке
    beginning = html.find('entry-date published updated')
    beginning = html.find('>', beginning)+1
    end = html.find('<', beginning)
    date = html[beginning:end].split('.')
    beginning = html.find('<title>') + len('<title>')
    end = html.find(' | Газета Йошкар-Ола', beginning)
    title = html[beginning:end]
    if html.find('<meta name="author" content="') != -1:
        beginning = html.find('<meta name="author" content="')+len('<meta name="author" content="')
        end = html.find('"/>', beginning)
        author = html[beginning:end]
    else:
        author = 'None'
    pattern = '>([а-яА-Я ]*)[^а-яА-Я]*?>\s*?<h1'
    if re.search(pattern, html):
        topic = re.search(pattern, html).group(1)
    else:
        topic = 'None'
    directory = root + os.sep + 'plain' + os.sep 
    if len(date) != 3:
        directory += 'None' + os.sep
        date1 = 'None'
        date2 = 'None'
    else:
        directory += date[2] + os.sep + date[1] + os.sep
        date1 = '.'.join(date)
        date2 = date[2]
    if not os.path.exists(directory):
        os.makedirs(directory)
    f = open(directory + 'статья' + file_number + '.txt', 'w', encoding='utf-8')
    f.write('@au ' + author + '\n')
    f.write('@ti ' + title + '\n')
    f.write('@da ' + date1 + '\n')
    f.write('@topic' + topic + '\n')
    f.write('@url' + url + '\n')
    f.write(content)
    f.close()
    with open(tab, 'a', encoding='utf-8') as fm:
        row = '%s\t%s\t%s\t%s\tпублицистика\t%s\tнейтральный\tн-возраст\tн-уровень\tгородская\t%s\tГазета Йошкар-Ола\t%s\tгазета\tРоссия\tМарий Эл\tru\n' % (directory + 'статья' + file_number + '.txt', author, title, date1, topic, url, date2)
        fm.write(row)

In [10]:
craw_list = crawler()
tab = get_ready()
for number, url in enumerate(craw_list):
    page(url, str(number), tab)